### Цели исследования

Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Шаг 1. Загрузка данных

In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

### Шаг 2. Общая информация о данных.

Выведем первые строки в каждой таблице и посчитаем сколкьо всего строк.

In [5]:
# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
query = '''SELECT COUNT(*) 
FROM books '''
get_sql_data(query)

,count
0,1000


Данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

В таблице 1000 строк.

In [7]:
query1 = '''
SELECT * 
FROM authors 
LIMIT 5'''
get_sql_data(query1)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query1 = '''SELECT COUNT(*) 
FROM authors '''
get_sql_data(query1)

,count
0,636


Данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

В таблице 636 строк

In [9]:
query2 = '''SELECT * 
FROM publishers 
LIMIT 5'''
get_sql_data(query2)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
query2 = '''SELECT COUNT(*) 
FROM publishers '''
get_sql_data(query2)

,count
0,340


Данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

В таблице 340 строк.

In [11]:
query3 = '''SELECT * 
FROM ratings 
LIMIT 5'''
get_sql_data(query3)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
query3 = '''SELECT COUNT(*) 
FROM ratings '''
get_sql_data(query3)

,count
0,6456


Данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

В таблице 6456 строк.

In [13]:
query4 = '''SELECT * 
FROM reviews 
LIMIT 5'''
get_sql_data(query4)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
query4 = '''SELECT COUNT(*) 
FROM reviews '''
get_sql_data(query4)

,count
0,2793


Данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

В таблице 2793 строки.

### Шаг 3. Исследовательский анализ.

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [15]:
query5 = '''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
'''

In [16]:
get_sql_data(query5)

,count
0,819


После 1 января 2000 года вышло 819 книг.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [17]:
query6 = ''' WITH s3 AS
(WITH s1 AS (SELECT book_id, 
COUNT(review_id) AS review_count
FROM reviews
GROUP BY book_id),
s2 AS (SELECT book_id,
ROUND(AVG(rating), 2) AS avg_rating
FROM ratings
GROUP BY book_id)

SELECT s2.book_id,
s1.review_count,
s2.avg_rating

FROM 
s1 FULL JOIN s2 ON s1.book_id = s2.book_id
ORDER BY s2.book_id)

SELECT s3.book_id,
b.title,
s3.review_count,
s3.avg_rating
FROM s3 JOIN books AS b ON s3.book_id = b.book_id
'''

In [18]:
get_sql_data(query6)

,book_id,title,review_count,avg_rating
0,1,'Salem's Lot,2.0,3.67
1,2,1 000 Places to See Before You Die,1.0,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.67
3,4,1491: New Revelations of the Americas Before C...,2.0,4.50
4,5,1776,4.0,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.67
996,997,Xenocide (Ender's Saga #3),3.0,3.40
997,998,Year of Wonders,4.0,3.20
998,999,You Suck (A Love Story #2),2.0,4.50


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [19]:
query7 = '''
SELECT *
FROM publishers
WHERE publisher_id IN (SELECT publisher_id
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY COUNT(book_id) DESC
LIMIT 1)

'''

In [20]:
get_sql_data(query7)

,publisher_id,publisher
0,212,Penguin Books


Издательство Penguin Books выпустило наибольшее число книг.

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [27]:
query8= '''
WITH s1 AS (SELECT b.author_id,
AVG(r.rating)
FROM ratings AS r
JOIN books AS b ON r.book_id = b.book_id
WHERE r.book_id IN (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id)>50)
GROUP BY b.author_id
ORDER BY avg DESC)
SELECT s1.author_id,
a.author
FROM s1 JOIN authors AS a ON s1.author_id = a.author_id
ORDER BY (avg) DESC
LIMIT 1
'''

In [28]:
get_sql_data(query8)

,author_id,author
0,236,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [23]:
query9 = '''
WITH 
s1 AS (SELECT username,
(COUNT(review_id))
FROM reviews
WHERE username IN (SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT(rating_id) > 48)
GROUP BY username )
SELECT AVG(count)
FROM s1 
'''

In [24]:
get_sql_data(query9)

,avg
0,24.0


В среднем пользователи, которые поставили больше 48 оценок давали 24 обзора.

### Шаг 4. Вывод

1. После 1 января 2000 года вышло 819 книг.
2. Для каждой книги посчитаны количество обзоров и средняя оценка
3. Издательство Penguin Books выпустило наибольшее число книг.
4. J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг.
5. В среднем пользователи, которые поставили больше 48 оценок давали 24 обзора.